### Importação dos pacotes

In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, accuracy_score
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf

import warnings
warnings.filterwarnings("ignore")

### Pré-processamento nos dados

In [2]:
# lendo os dados de um arquivo csv
dataframe = pd.read_csv('/content/drive/MyDrive/vinbigdata/train.csv')
# criando uma coluna com os caminhos relativos as imagens
dataframe['image_path'] = '/content/drive/MyDrive/vinbigdata_seg/' + dataframe.image_id + '.jpg'

In [3]:
print('total de imagens disponíveis:', str(len(set(dataframe['image_path']))))

total de imagens disponíveis: 15000


In [4]:
# visualizando os casos disponíveis
dataframe['class_name'].value_counts()

No finding            31818
Aortic enlargement     7162
Cardiomegaly           5427
Pleural thickening     4842
Pulmonary fibrosis     4655
Nodule/Mass            2580
Lung Opacity           2483
Pleural effusion       2476
Other lesion           2203
Infiltration           1247
ILD                    1000
Calcification           960
Consolidation           556
Atelectasis             279
Pneumothorax            226
Name: class_name, dtype: int64

In [5]:
# removendo os casos não relativos a distúrbios pulmonares
dataframe = dataframe[dataframe.class_name != 'Aortic enlargement']
dataframe = dataframe[dataframe.class_name != 'Cardiomegaly']
dataframe = dataframe[dataframe.class_name != 'Other lesion']
dataframe = dataframe[dataframe.class_name != 'Consolidation']

In [6]:
# separando os casos rotulados como normais e anormais
normal_cases = dataframe[(dataframe.class_id == 14) & (dataframe.class_name == 'No finding')]
abnormal_cases = dataframe[(dataframe.class_id != 14) & (dataframe.class_name != 'No finding')]

print('total de dados após a filtração:', str(len(set(normal_cases['image_path'])) + len(set(abnormal_cases['image_path']))))

total de dados após a filtração: 13948


In [7]:
# removendo as imagens repetidas
normal_data = normal_cases[['image_path', 'class_name']].drop_duplicates(subset = 'image_path', )
abnormal_data = abnormal_cases[['image_path', 'class_name']].drop_duplicates(subset = 'image_path', )

# criando dataframes especifos com caminhos para as imagens e rótulos
normal_data['target'] = 'normal'
abnormal_data['target'] = 'abnormal'

In [8]:
print('quantidade de dados rotulados como normais:', len(normal_data))
print('quantidade de dados rotulados como anormais:', len(abnormal_data))

quantidade de dados rotulados como normais: 10606
quantidade de dados rotulados como anormais: 3342


In [9]:
# removendo 69% dos casos normais para balancear os dados
normal, _ = train_test_split(normal_data, test_size = 0.69, random_state = 42)

In [10]:
print('quantidade de dados rotulados como normais:', len(normal))
print('quantidade de dados rotulados como anormais:', len(abnormal_data))

quantidade de dados rotulados como normais: 3287
quantidade de dados rotulados como anormais: 3342


In [11]:
# concatenando os dataframes de casos normais e anormais
full_data = pd.concat([normal, abnormal_data])

In [12]:
# misturando todos os dados do dataframe e reiniciando os valores dos índices 
full_data = full_data.sample(frac = 1, axis = 0, random_state = 42).reset_index(drop=True)

In [13]:
full_data = full_data[full_data.class_name != 'No finding']
full_data = full_data[full_data.class_name != 'Atelectasis']
full_data = full_data[full_data.class_name != 'Pneumothorax']

In [14]:
full_data['class_name'].value_counts()

Pleural thickening    908
Pulmonary fibrosis    746
Lung Opacity          448
Nodule/Mass           347
Pleural effusion      332
Infiltration          169
Calcification         167
ILD                   152
Name: class_name, dtype: int64

In [15]:
# separando os dados de treinamento e de teste
train_df, test_df = train_test_split(full_data, stratify = full_data['class_name'],
                                     test_size = 0.2, random_state = 42)

In [16]:
# separando os dados de validação dos dados de treinamento
train_df, validation_df = train_test_split(train_df, stratify = train_df['class_name'],
                                           test_size = 0.2, random_state = 42)

In [17]:
# visualizando a quantidade de dados
print('quantidade de imagens de treinamento:', len(train_df['image_path']))
print('quantidade de rótulos de treinamento:', len(train_df['class_name']))
print('quantidade de imagens de teste:', len(test_df['image_path']))
print('quantidade de rótulos de teste:', len(test_df['class_name']))
print('quantidade de imagens de validação:', len(validation_df['image_path']))
print('quantidade de rótulos de validação:', len(validation_df['class_name']))

quantidade de imagens de treinamento: 2092
quantidade de rótulos de treinamento: 2092
quantidade de imagens de teste: 654
quantidade de rótulos de teste: 654
quantidade de imagens de validação: 523
quantidade de rótulos de validação: 523


In [18]:
# organizando um dicionário para realizar o balanceamento nos dados das classes
class_weights = class_weight.compute_class_weight('balanced', np.unique(train_df['class_name']),
                                                  train_df['class_name'])
class_weight = {0: class_weights[0], 1: class_weights[1], 2: class_weights[2], 
                3: class_weights[3], 4: class_weights[4], 5: class_weights[5],
                6: class_weights[6], 7: class_weights[7]}

In [19]:
# normalizando as imagens de treinamento e aplicando aumento de dados
image_generator = ImageDataGenerator()

# criando o gerador de imagens de treinamento 
train_generator = image_generator.flow_from_dataframe(
                                                      dataframe = train_df,
                                                      directory = '',
                                                      x_col = 'image_path',
                                                      y_col = 'class_name',
                                                      batch_size = 32,
                                                      seed = 42,
                                                      shuffle = True,
                                                      class_mode = 'categorical',
                                                      target_size = (256, 256))
# criando o gerador de imagens de validação 
valid_generator = image_generator.flow_from_dataframe(
                                                      dataframe = validation_df,
                                                      directory = '.', 
                                                      x_col = 'image_path',
                                                      y_col = 'class_name',
                                                      batch_size = 32,
                                                      seed = 42,
                                                      shuffle = True,
                                                      class_mode = 'categorical',
                                                      target_size = (256, 256))

# normalizando as imagens de teste 
test_datagen = ImageDataGenerator()

test_generator = test_datagen.flow_from_dataframe(
                                                  dataframe = test_df, 
                                                  directory = '.',
                                                  x_col = 'image_path',
                                                  y_col = 'class_name',
                                                  batch_size = 32,
                                                  seed = 42,
                                                  shuffle = True,
                                                  class_mode = 'categorical',
                                                  target_size = (256, 256))

Found 2092 validated image filenames belonging to 8 classes.
Found 523 validated image filenames belonging to 8 classes.
Found 654 validated image filenames belonging to 8 classes.


### Preparando a rede neural convolucional

In [20]:
# baixando os pesos treinados da rede inception
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

--2021-05-07 02:27:07--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.70.128, 74.125.132.128, 74.125.201.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.70.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M  62.3MB/s    in 1.3s    

2021-05-07 02:27:08 (62.3 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [21]:
# referenciando o local em que os pesos estão armazenados
local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

# carregando a arquitetura inception pré-treinada
pre_trained_model = InceptionV3(input_shape = (256, 256, 3), 
                                include_top = False, 
                                weights = None)

# carregando os pesos treinados com outros dados 
pre_trained_model.load_weights(local_weights_file)

# definindo as flags iniciais  
#pre_trained_model.trainable = True
#set_trainable = False

# para a arquitetura inception, a rede será retreinada a partir da camada 'mixed8'
#for layer in pre_trained_model.layers:
#    if layer.name == 'mixed8':
#        set_trainable = True
#    if set_trainable:
#        layer.trainable = True
#    else:
#       layer.trainable = False

# visualizando a arquitetura definida
#pre_trained_model.summary()

# obtendo a última camada como sendo a nomeada por 'mixed7'
last_layer = pre_trained_model.get_layer('mixed7')
last_output = last_layer.output

In [22]:
# definindo uma camada de achatamento
x = layers.Flatten()(last_output)
# conecatando a rede uma camada com 1024 neurônios e função de ativação relu
x = layers.Dense(units = 512, activation = 'relu')(x)     
# conecatando a rede uma camada com 128 neurônios e função de ativação relu
x = layers.Dense(units = 256, activation = 'relu')(x) 
# aplicando uma camada de dropout com uma taxa de 20% (normalização)
x = layers.Dropout(rate = 0.2)(x)                  
# adicionando uma camada de saída com um neurônio e uma função de ativação sigmoide
x = layers.Dense  (units = 8, activation = 'softmax')(x)           

# conecatando as camadas definidas acima com a arquitetura inception
model = Model(pre_trained_model.input, x) 

# compilando a rede 
model.compile(optimizer = optimizers.RMSprop(learning_rate = 0.0001), loss = 'categorical_crossentropy', 
              metrics = ['acc'])

In [23]:
# definindo o caminho pelo qual os pesos serão armazenados 
filepath = "transferlearning_weights.hdf5"
# callback para salvar o melhor valor dos pesos em relação ao desempenho com os dados de validação 
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc', verbose = 1, save_best_only = True, mode = 'max')

In [24]:
# definindo um array de callbacks
callbacks = [checkpoint]

In [25]:
# treinando a rede neural convolucional
history = model.fit_generator(train_generator, steps_per_epoch = 2092 // 32, 
                              validation_data = valid_generator, validation_steps = 523 // 32,
                              callbacks = callbacks, epochs = 20, class_weight = class_weight)

Epoch 1/20
65/65 [==============================] - 156s 2s/step - loss: 2.3938 - acc: 0.1328 - val_loss: 2.0318 - val_acc: 0.0781

Epoch 00001: val_acc improved from -inf to 0.07812, saving model to transferlearning_weights.hdf5
Epoch 2/20
65/65 [==============================] - 148s 2s/step - loss: 1.8938 - acc: 0.2239 - val_loss: 2.3133 - val_acc: 0.1406

Epoch 00002: val_acc improved from 0.07812 to 0.14062, saving model to transferlearning_weights.hdf5
Epoch 3/20
65/65 [==============================] - 149s 2s/step - loss: 1.2813 - acc: 0.4531 - val_loss: 2.2203 - val_acc: 0.3047

Epoch 00003: val_acc improved from 0.14062 to 0.30469, saving model to transferlearning_weights.hdf5
Epoch 4/20
65/65 [==============================] - 149s 2s/step - loss: 0.6473 - acc: 0.7144 - val_loss: 2.5870 - val_acc: 0.2285

Epoch 00004: val_acc did not improve from 0.30469
Epoch 5/20
65/65 [==============================] - 149s 2s/step - loss: 0.2669 - acc: 0.8627 - val_loss: 3.5858 - val_acc

### Salvando o modelo

In [26]:
# carregando o melhor peso obtido para o modelo
best_model = model
best_model.load_weights('/content/transferlearning_weights.hdf5')

In [27]:
# salvando os dois modelos obtidos durante o treinamento
model.save('model1')
best_model.save('model2')

INFO:tensorflow:Assets written to: model1/assets
INFO:tensorflow:Assets written to: model2/assets


In [28]:
best_model.evaluate(test_generator)

21/21 [==============================] - 8s 361ms/step - loss: 4.3026 - acc: 0.3287


[4.302553176879883, 0.32874616980552673]

### Métricas de avaliação do modelo

In [29]:
# carregando os dados de teste
for i in range(0, 21):
  (x1, y1) = test_generator[i]
  if i == 0:
    x, y = x1, y1
  else:
    x = np.concatenate((x, x1))
    y = np.concatenate((y, y1))

In [30]:
# realizando a predição para os dados de teste
predict = model.predict(x)

count_global = 0
for predicts in predict:
    count = 0
    aux = np.zeros((8,))
    for values in predicts:
        if values >= 0.50:
            aux[count] = 1.
        else:
            aux[count] = 0.
        count += 1
    predict[count_global] = aux
    count_global += 1

In [31]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, confusion_matrix
print('Matriz de Confusão:\n', confusion_matrix(y.argmax(axis = 1), predict.argmax(axis = 1)))
print('Acurácia:', accuracy_score(y.argmax(axis = 1), predict.argmax(axis = 1)))
print('Precisão', precision_score(y.argmax(axis = 1), predict.argmax(axis = 1), average = 'weighted'))
print('Sensibilidade:', recall_score(y.argmax(axis = 1), predict.argmax(axis = 1), average = 'weighted')) 
print('F1_Score:', f1_score(y.argmax(axis = 1), predict.argmax(axis = 1), average = 'weighted'))

Matriz de Confusão:
 [[  3   0   0   0   0   0  16  14]
 [  1   1   0   2   1   0  14  11]
 [  1   0   0   1   1   2  12  17]
 [  8   0   0   4   5   8  41  24]
 [  4   0   0   1   1   2  31  31]
 [  3   0   0   4   2  18  29  10]
 [  8   0   0   2   2   7 119  44]
 [  7   0   0   3   5   5  64  65]]
Acurácia: 0.32262996941896027
Precisão 0.30226597728806326
Sensibilidade: 0.32262996941896027
F1_Score: 0.26532358658697824


In [33]:
train_generator.class_indices

{'Calcification': 0,
 'ILD': 1,
 'Infiltration': 2,
 'Lung Opacity': 3,
 'Nodule/Mass': 4,
 'Pleural effusion': 5,
 'Pleural thickening': 6,
 'Pulmonary fibrosis': 7}